In [ ]:
!pip install fastapi graphene starlette-graphene3 uvicorn

In [ ]:
from fastapi import FASTAPI 
from starlette_graphene3 import GraphQLApp
import graphene

# define graphQL schema
class User(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()


# in-memory mock database
users = {
    '1': {'id': '1', 'name': 'Alice'},
    '2': {'id': '2', 'name': 'Bob'}
    }

# query: read-only
class Query(graphene.ObjectType):
    user = graphene.Field(User, id=graphene.ID(requires=True))

    def resolve_user(root, info, id):
        return users.get(id)

# mutation: write/update
class CreateUser(graphene.Mutation):
    class Arguments:
        id = graphene.ID()
        name = graphene.String()

    ok = graphene.Boolean()
    user = graphene.Field(User)

    def mutate(root, info, id, name):
        user = {'id':id, 'name':name}
        users[id] = user
        return CreateUser(user=user, ok=True)



class Mutation(graphene.ObjectType):
    create_user = CreateUser.Field()



# assemble scheme
app = FastAPI()
app.add_route('/graphql', GraphQLApp(schema=graphene.Schema(query=Query, mutation=Mutation)))
